In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import nltk
from collections import Counter
from torch.utils.data import Dataset, DataLoader


nltk.download('punkt')

# -----------------------
# Data Preparation
# -----------------------

# Provided dataset: each tuple is (English, French)
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur"),
    ("They ride bicycles", "Ils font du vélo"),
    ("The coffee is hot", "Le café est chaud"),
    ("She wears glasses", "Elle porte des lunettes"),
    ("We visit our grandparents", "Nous rendons visite à nos grands-parents"),
    ("He plays the guitar", "Il joue de la guitare"),
    ("They go shopping", "Ils font du shopping"),
    ("The teacher explains the lesson", "Le professeur explique la leçon"),
    ("She takes the train to work", "Elle prend le train pour aller au travail"),
    ("We bake cookies", "Nous faisons des biscuits"),
    ("He washes his hands", "Il se lave les mains"),
    ("They enjoy the sunset", "Ils apprécient le coucher du soleil"),
    ("The river flows calmly", "La rivière coule calmement"),
    ("She feeds the cat", "Elle nourrit le chat"),
    ("We visit the museum", "Nous visitons le musée"),
    ("He fixes his bicycle", "Il répare son vélo"),
    ("They paint the walls", "Ils peignent les murs"),
    ("The baby sleeps peacefully", "Le bébé dort paisiblement"),
    ("She ties her shoelaces", "Elle attache ses lacets"),
    ("We climb the stairs", "Nous montons les escaliers"),
    ("He shaves in the morning", "Il se rase le matin"),
    ("They set the table", "Ils mettent la table"),
    ("The airplane takes off", "L'avion décolle"),
    ("She waters the plants", "Elle arrose les plantes"),
    ("We practice yoga", "Nous pratiquons le yoga"),
    ("He turns off the light", "Il éteint la lumière"),
    ("They play video games", "Ils jouent aux jeux vidéo"),
    ("The soup smells delicious", "La soupe sent délicieusement bon"),
    ("She locks the door", "Elle ferme la porte à clé"),
    ("We enjoy a picnic", "Nous profitons d'un pique-nique"),
    ("He checks his email", "Il vérifie ses emails"),
    ("They go to the gym", "Ils vont à la salle de sport"),
    ("The moon shines brightly", "La lune brille intensément"),
    ("She catches the bus", "Elle attrape le bus"),
    ("We greet our neighbors", "Nous saluons nos voisins"),
    ("He combs his hair", "Il se peigne les cheveux"),
    ("They wave goodbye", "Ils font un signe d'adieu")
]

# For English-to-French, the source is English and the target is French.
english_sentences = [pair[0] for pair in english_to_french]
french_sentences = [pair[1] for pair in english_to_french]
# Add special tokens to French target sentences.
french_sentences = ['<sos> ' + sent + ' <eos>' for sent in french_sentences]

# Simple tokenization using NLTK.
def tokenize(text):
    return nltk.word_tokenize(text.lower())

# Build vocabulary function 
def build_vocab(sentences):
    counter = Counter()
    for sent in sentences:
        counter.update(tokenize(sent))
    vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    for word, _ in counter.items():
        if word not in vocab:
            vocab[word] = len(vocab)
    return vocab

# Build vocabularies for English (source) and French (target)
eng_vocab = build_vocab(english_sentences)
fr_vocab = build_vocab(french_sentences)
eng_pad_idx = eng_vocab['<pad>']
fr_pad_idx = fr_vocab['<pad>']

# Inverse mapping for target decoding (French)
inv_fr_vocab = {i: w for w, i in fr_vocab.items()}

# Convert sentences to sequences of indices
def numericalize(sentences, vocab):
    return [[vocab.get(word, vocab['<unk>']) for word in tokenize(sent)] for sent in sentences]

eng_sequences = numericalize(english_sentences, eng_vocab)
fr_sequences = numericalize(french_sentences, fr_vocab)

# -----------------------
# Create Dataset and DataLoader
# -----------------------

def pad_seq(seq, max_len, pad_idx):
    return seq + [pad_idx] * (max_len - len(seq))

max_eng_len = max(len(seq) for seq in eng_sequences)
max_fr_len = max(len(seq) for seq in fr_sequences)

eng_sequences = [pad_seq(seq, max_eng_len, eng_pad_idx) for seq in eng_sequences]
fr_sequences = [pad_seq(seq, max_fr_len, fr_pad_idx) for seq in fr_sequences]

data = list(zip(eng_sequences, fr_sequences))
random.shuffle(data)
split_idx = int(0.8 * len(data))
train_data = data[:split_idx]
val_data = data[split_idx:]

class TranslationDataset(Dataset):
    def __init__(self, data):
        self.data = data  # list of (source, target)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        src, tgt = self.data[idx]
        return torch.tensor(src, dtype=torch.long), torch.tensor(tgt, dtype=torch.long)

train_dataset = TranslationDataset(train_data)
val_dataset = TranslationDataset(val_data)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# -----------------------
# Model Definitions (Encoder, Decoder, Seq2Seq) Without Attention
# -----------------------

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_idx)
        self.gru = nn.GRU(emb_dim, hid_dim, batch_first=True)
    def forward(self, src):
        embedded = self.embedding(src)  # [batch, src_len, emb_dim]
        _, hidden = self.gru(embedded)   # hidden: [1, batch, hid_dim]
        return hidden.squeeze(0)         # [batch, hid_dim]

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=pad_idx)
        self.gru = nn.GRU(emb_dim, hid_dim, batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
    def forward(self, input, hidden):
        # input: [batch] -> [batch, 1]
        input = input.unsqueeze(1)
        embedded = self.embedding(input)  # [batch, 1, emb_dim]
        output, hidden = self.gru(embedded, hidden.unsqueeze(0))  # output: [batch, 1, hid_dim]
        hidden = hidden.squeeze(0)         # [batch, hid_dim]
        prediction = self.fc_out(output.squeeze(1))  # [batch, output_dim]
        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        tgt_len = tgt.size(1)
        tgt_vocab_size = self.decoder.fc_out.out_features
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        hidden = self.encoder(src)  # [batch, hid_dim]
        input = tgt[:, 0]           # first token (<sos>)
        for t in range(1, tgt_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = tgt[:, t] if teacher_force else top1
        return outputs

# Hyperparameters and device
INPUT_DIM = len(eng_vocab)
OUTPUT_DIM = len(fr_vocab)
EMB_DIM = 256
HID_DIM = 512
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, eng_pad_idx)
decoder = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, fr_pad_idx)
model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

# -----------------------
# Training Setup
# -----------------------

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=fr_pad_idx)

def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0
    for src, tgt in dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        optimizer.zero_grad()
        output = model(src, tgt, teacher_forcing_ratio=0.5)
        # output: [batch, tgt_len, output_dim]
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        preds = output.argmax(1)
        non_pad = (tgt != fr_pad_idx)
        correct = (preds == tgt) & non_pad
        epoch_correct += correct.sum().item()
        epoch_total += non_pad.sum().item()
    return epoch_loss / len(dataloader), epoch_correct / epoch_total

def evaluate_epoch(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0
    with torch.no_grad():
        for src, tgt in dataloader:
            src = src.to(DEVICE)
            tgt = tgt.to(DEVICE)
            output = model(src, tgt, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)
            loss = criterion(output, tgt)
            epoch_loss += loss.item()
            preds = output.argmax(1)
            non_pad = (tgt != fr_pad_idx)
            correct = (preds == tgt) & non_pad
            epoch_correct += correct.sum().item()
            epoch_total += non_pad.sum().item()
    return epoch_loss / len(dataloader), epoch_correct / epoch_total

N_EPOCHS = 50
for epoch in range(1, N_EPOCHS + 1):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate_epoch(model, val_loader, criterion)
    print(f"Epoch {epoch}/{N_EPOCHS}:")
    print(f"  Training Loss: {train_loss:.4f} | Training Accuracy: {train_acc:.4f}")
    print(f"  Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_acc:.4f}")

# -----------------------
# Inference / Qualitative Validation
# -----------------------

def translate_sentence(model, sentence, eng_vocab, fr_vocab, inv_fr_vocab, max_eng_len, max_fr_len):
    model.eval()
    tokens = tokenize(sentence)
    numericalized = [eng_vocab.get(tok, eng_vocab['<unk>']) for tok in tokens]
    numericalized = numericalized + [eng_pad_idx] * (max_eng_len - len(numericalized))
    src_tensor = torch.tensor(numericalized, dtype=torch.long).unsqueeze(0).to(DEVICE)
    hidden = model.encoder(src_tensor)
    input_token = torch.tensor([fr_vocab['<sos>']], dtype=torch.long).to(DEVICE)
    decoded_sentence = []
    for t in range(max_fr_len):
        output, hidden = model.decoder(input_token, hidden)
        top1 = output.argmax(1).item()
        if top1 == fr_vocab['<eos>']:
            break
        decoded_sentence.append(inv_fr_vocab.get(top1, '<unk>'))
        input_token = torch.tensor([top1], dtype=torch.long).to(DEVICE)
    return ' '.join(decoded_sentence)

# Test translation on some English sentences.
test_sentences = [
    "I am cold",
    "She speaks French fluently",
    "We love music",
    "He listens to music while jogging",
    "They travel around the world"
]

for sentence in test_sentences:
    translation = translate_sentence(model, sentence, eng_vocab, fr_vocab, inv_fr_vocab, max_eng_len, max_fr_len)
    print(f"English: {sentence}")
    print(f"Translated French: {translation}")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahosain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1/50:
  Training Loss: 4.7661 | Training Accuracy: 0.2477
  Validation Loss: 3.5886 | Validation Accuracy: 0.2907
Epoch 2/50:
  Training Loss: 3.1835 | Training Accuracy: 0.4053
  Validation Loss: 3.6555 | Validation Accuracy: 0.2643
Epoch 3/50:
  Training Loss: 2.8323 | Training Accuracy: 0.4201
  Validation Loss: 3.2897 | Validation Accuracy: 0.3833
Epoch 4/50:
  Training Loss: 2.5054 | Training Accuracy: 0.5023
  Validation Loss: 3.8033 | Validation Accuracy: 0.3612
Epoch 5/50:
  Training Loss: 2.7447 | Training Accuracy: 0.3995
  Validation Loss: 3.4108 | Validation Accuracy: 0.3128
Epoch 6/50:
  Training Loss: 2.4202 | Training Accuracy: 0.4658
  Validation Loss: 3.4336 | Validation Accuracy: 0.4361
Epoch 7/50:
  Training Loss: 2.2476 | Training Accuracy: 0.5525
  Validation Loss: 3.7211 | Validation Accuracy: 0.4053
Epoch 8/50:
  Training Loss: 2.0958 | Training Accuracy: 0.5753
  Validation Loss: 3.5183 | Validation Accuracy: 0.4361
Epoch 9/50:
  Training Loss: 1.9698 | Tr